In [ ]:
import time

import numpy as np

from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils import data

from torchvision import datasets
from torchvision import transforms

- Task 1.1:  5 points
- Task 1.2:  5 points
- Task 1.3: 10 points
- Task 1.4:  5 points
- Task 1.5: 10 points
- Task 2.1: 10 points
- Task 2.2: 10 points
- Task 2.3: 20 points
- Task 3.1:  5 points
- Task 3.2: 10 points
- Task 4.1: 10 points

# 1. Attention is All You Need
Read this paper **Attention is All You Need** (https://arxiv.org/pdf/1706.03762.pdf)
![Attention](images/attention.png)
### 1.1 Scaled Dot Product Attention
$$\mathrm{Attention}(Q, K, V) = \mathrm{softmax}(\frac{QK^\intercal}{\sqrt{d_k}})V,$$
where
$Q\in\mathbb{R}^{N_q\times d_{k}}$, $K\in\mathbb{R}^{N_k\times d_{k}}$, and $V\in\mathbb{R}^{N_k\times d_{v}}$.
#### Make sure you do understand the followings:
1. Why do we design the matrix dimensions of Q, K and V as above?
2. Which axis do you speicfy when using the function **softmax**?
3. Why do we need to scale the dot products with $\frac{1}{\sqrt{d_k}}$

### 1.2 Multi-head Attention
$$\mathrm{MultiHead}(Q, K, V)=\mathrm{Concat}(\mathrm{head}_1, \mathrm{head}_2, \dots, \mathrm{head}_h)W^{O},$$
where,
$$\mathrm{head}_i=\mathrm{Attention}(QW^Q_i, KW^K_i, VW^V_i),$$
$Q\in\mathbb{R}^{N_q\times d_{\text{model}}}$, $K\in\mathbb{R}^{N_k\times d_{\text{model}}}$, $V\in\mathbb{R}^{N_k\times d_{\text{model}}}$, $W^Q_i\in\mathbb{R}^{d_{\text{model}}\times d_{\text{head}}}$, $W^K_i\in\mathbb{R}^{d_{\text{model}}\times d_{\text{head}}}$, $W^V_i\in\mathbb{R}^{d_{\text{model}}\times d_{\text{head}}}$, and $W^O\in\mathbb{R}^{d_{\text{model}}\times d_{\text{model}}}$.

In practice, we often use multi-head attention. We make the following modifications based on scaled dot product attention,
1. We linearly project Q, K and V first. This step introduces **learnable** parameters to the model.
2. We often set the dimensions of Q, K and V the same, i.e., $d_v=d_k=d=d_\text{model}$
3. We do multiple small attentions in parallel. Assuming we have $h$ small attention models (a.k.a, heads). Each head uses dimension of $d = d_{\text{head}}=\frac{d_\text{model}}{h}$.
4. We concatenate the outputs from all heads.

### 1.3 Questions
Can you re-write the function **Attention()** in vector form? The inputs are rows of Q, K and V. For example, we can write $Q$ as vectors, $Q=\{q_i\in\mathbb{R}^{d_k}\}^{N_q}_{i}$

In [ ]:
#### Implement the function Attention(Q, K, V)

class Attention(nn.Module):
    """
    Scaled Dot-Product Attention in the paper "Attention Is All You Need"
    """
    def __init__(self):
        super(Attention, self).__init__()

    def forward(self, query, key, value):
        # query: B x N_q x d_k
        # key: B x N_k x d_k
        # value: B x N_k x d_v
        
        assert query.shape[2] == key.shape[2] and key.shape[1] == value.shape[1]
        
        #### Task 1.1
        #### Scaled Dot Product Attention
        scale = None
        attn = None
        x = None
        ####
        
        return x, attn # B x N_q x d_v, B x N_q x N_k

In [ ]:
#### Implement the multi-head attention which performs multiple attention sequentially.
#### 1. Perform multiple attentions one by one
#### 2. Concatenate all the outputs
#### 3. Linear project

class SequentialMultiHead(nn.Module):
    """
    Multi-Head Attention in the paper "Attention Is All You Need"
    """
    def __init__(self, d_model, num_heads):
        super(SequentialMultiHead, self).__init__()

        assert d_model % num_heads == 0

        self.d_head = int(d_model / num_heads)
        self.num_heads = num_heads
        
        self.scaled_dot_attention = Attention()

        #### Task 1.2
        #### Sequential Multi Head Attention
        self.query_proj = None # W^Q
        self.key_proj = None # W^K
        self.value_proj = None # W^V
        self.output_proj = None # W^O
        ####
        
    def forward(self, query, key, value):
        # query: B x N_q x D
        # key: B x N_k x D
        # value: B x N_k x D
        assert query.shape[2] == key.shape[2] and query.shape[2] == value.shape[2]
        
        #### Task 1.2
        #### Sequential Multi Head Attention
        outputs = None
        ####
        
        return outputs

In [ ]:
#### Implement the multi-head attention which performs multiple attention in parallel.
#### This is a faster version of the multi-head attention
#### Hints:
#### Merge the two axis: batch size and head

class ParallelMultiHead(nn.Module):
    """
    Multi-Head Attention in the paper "Attention Is All You Need"
    """
    def __init__(self, d_model, num_heads):
        super(ParallelMultiHead, self).__init__()

        assert d_model % num_heads == 0

        self.d_head = int(d_model / num_heads)
        self.num_heads = num_heads
        
        self.scaled_dot_attention = Attention()
        
        self.query_proj = nn.Linear(d_model, self.d_head * num_heads)
        self.key_proj = nn.Linear(d_model, self.d_head * num_heads)
        self.value_proj = nn.Linear(d_model, self.d_head * num_heads)

        #### Task 1.3
        #### Parallel Multi Head Attention
        self.query_proj = None # W^Q
        self.key_proj = None # W^K
        self.value_proj = None # W^V
        self.output_proj = None # W^O
        ####
        
    def forward(self, query, key, value):
        # query: B x N_q x D
        # key: B x N_k x D
        # value: B x N_k x D
        assert query.shape[2] == key.shape[2] and query.shape[2] == value.shape[2] and key.shape[1] == value.shape[1]
        
        batch_size = value.size(0)

        #### Task 1.3
        #### Parallel Multi Head Attention
        outputs = None
        ####
        
        return outputs

### 1.4 Self Attention Block
![Block](images/layer.png)
When Q, K and V are the same data, the attention becomes the self-attention. To build a layers
Given the data $X\in\mathrm{R}^{N \times d_{\text{model}}}$, the self-attention is done as follows,
$$X \leftarrow \mathrm{Norm}_1(X+\mathrm{MultiHead}(X, X, X)),$$
$$X\leftarrow \mathrm{Norm}_2(X+\mathrm{FFN}(X)),$$
where $\mathrm{FFN}$ is feed-forward network which consists two fully-connected layers,
$$\mathrm{FFN}(x) = \mathrm{FC}_2(\mathrm{ReLU}(\mathrm{FC}_1(x))),$$
where $x$ is a row in the matrix $X$.

In [ ]:
#### Implement the feed-forward network.
#### It is a two-layered MLP.
#### Usually the inner (hidden) dimension is 4 time larger.

class FFN(nn.Module):
    def __init__(self, d_model, d_inner):
        super().__init__()

        #### Task 1.4
        #### Feed-forward network
        #### Define your network here
        ####


    def forward(self, x):
        #### Task 1.4
        #### Feed-forward network
        x = None
        ####
        return x

In [ ]:
#### Implement the self attention block.

class SelfAttentionBlock(nn.Module):

    def __init__(
            self,
            d_model,
            num_heads,
            mlp_ratio=4.,
    ):
        super().__init__()

        #### Task 1.5
        #### Self Attention Block
        #### Define your network here
        #### Use your previously defined ParallelMultiHead and FFN
        
    def forward(self, x):
        
        #### Task 1.5
        #### Feed-forward network
        #### Be cautious about the exceuting order.
        x = None
        ####
        
        return x

# 2 Vision Transformers
Read the paper **Vision Transfomers** (ViT) (https://arxiv.org/abs/2010.11929).
![ViT](images/vit_figure.png)

In [ ]:
### Read the paper ViT section 3.1.
### 1. Split an image into non-overlapping patches
### 2. Apply a shared fully-connected layer to all patches
### With this function, we can turn inputs of size B x 3 x H x W into B x P^2 x D

class PatchEmbed(nn.Module):
    def __init__(
            self,
            image_size=32,
            patch_size=4,
            input_channels=3,
            d_model=256,
    ):
        super().__init__()
        
        assert image_size % patch_size == 0
        
        self.image_size = (image_size, image_size)
        self.patch_size = (patch_size, patch_size)

        #### Task 2.1
        #### Patch Embedding
        #### Define your network here
        ####

    def forward(self, x):
        B, C, H, W = x.shape
        
        assert H == self.image_size[0] and W == self.image_size[1]

        #### Task 2.1
        #### Feed-forward network
        #### Be cautious about the exceuting order.
        x = None
        ####
        
        return x # B x P^2 x D

In [ ]:
#### Re-define another self attention block
#### Note that the order of normalization and skip connection is differnt

class SelfAttentionBlock(nn.Module):

    def __init__(
            self,
            d_model,
            num_heads,
            mlp_ratio=4,
    ):
        super().__init__()

        #### Task 2.2
        #### Self Attention Block
        #### Define your network here
        ####

    def forward(self, x):
        #### Task 2.2
        #### Self Attention Block
        x = None
        ####
        
        return x

In [ ]:
#### Look at the main figure of this paper
#### You need the followings:
#### 1. A PatchEmbed to convert an image to a sequence of vectors
#### 2. A classification token which is used for classifiation (implement it with nn.Parameter)
#### 3. Learnable positional embeddings (you can use nn.Embedding in pytorch). How many vectors do you need?
#### 4. A series of self-attention blocks.
#### 5. A classification head (we use it to project the classification token to logits)

class VisionTransformer(nn.Module):
    def __init__(
        self,
        image_size=32, # image size 32 x 32
        patch_size=4, # patch size 4 x 4
        input_channels=3, # image channels (R, G, B)
        num_classes=10, # cifar10 has 10 classes
        d_model=256,
        depth=4, # we use 4 self attention blocks here
        num_heads=8,
        mlp_ratio=4,
    ):
        super().__init__()

        #### Task 2.3
        #### Vision Transformer
        #### Following the hints above
        #### Define your network here
        ####
        
    def forward(self, x):
        #### Task 2.3
        #### Vision Transformer
        x = None
        ####
        return x

# 3 Training a ViT for image classifiation
Here we train a model for CIFAR10 classification. You might need some code in your previous projects.

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        #### Task 3.1
        #### Data augmentation
        ####
    ]),
    'test': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
}


train_dataset = datasets.CIFAR10(root='./data', train=True, transform=data_transforms['train'], download=True)
train_dataloader = data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=1)

test_dataset = datasets.CIFAR10(root='./data', train=True, transform=data_transforms['test'], download=True)
test_dataloader = data.DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=1)

class_names = train_dataset.classes

device = torch.device("cuda:0")

In [ ]:
print(class_names)

In [ ]:
epochs = 50

#### Task 3.2
#### Training code
model = None
criterion = None
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.0001)
####

for epoch in range(epochs):
    
    since = time.time()
    
    running_loss = 0.0
    running_corrects = 0
            
    for inputs, labels in train_dataloader:
        
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        
        epoch_loss = running_loss / len(train_dataset)
        epoch_acc = running_corrects.double() / len(test_dataset)
        
    print('[Train] [Epoch: {}/{}] Loss: {:.4f} Acc: {:.4f}'.format(epoch, epochs, epoch_loss, epoch_acc))

    running_loss = 0.0
    running_corrects = 0
    
    with torch.no_grad():
        for inputs, labels in test_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(train_dataset)
            epoch_acc = running_corrects.double() / len(test_dataset)
            
#             if iters % 10 == 0:
        print('[Test ] [Epoch: {}/{}] Loss: {:.4f} Acc: {:.4f}'.format(epoch, epochs, epoch_loss, epoch_acc))
        
    time_elapsed = time.time() - since
    print('Epoch complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print()

#### Task 3.2
#### Training code
#### Save your network's checkpoint
####

In [ ]:
#### Visualize your predictions

import matplotlib.pyplot as plt

def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.5, 0.5, 0.5])
    std = np.array([0.5, 0.5, 0.5])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    
def display_predictions(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(test_dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
display_predictions(model, num_images=6)

# 4 Analysis
Here we are going to visualize the attention matrix. We will get something similar to the figure (taken from the ViT paper)
![Map](images/vs.png)

In [ ]:
#### Load images
def transfer_single_img_to_tensor(img_path):
    image = Image.open(img_path)
    img = np.asarray(image)/255
    mean = np.array([0.5, 0.5, 0.5])
    std = np.array([0.5, 0.5, 0.5])
    
    inp = (img - mean) / std
    inp = np.asarray(inp, dtype=np.float32)
    inp = inp.transpose((2, 0, 1))
    inp = np.expand_dims(inp, axis=0)
    inp = torch.from_numpy(inp, )
    inputs = inp.to(device)
    return image, inputs

In [ ]:
#### Task 4.1
#### Attention matrix
#### Make sure you saved the trained network before this step!!!!
model = None
model.load_state_dict(None)
####

attentions = []

def get_attn_softmax(name):
    def hook(model, input, output):
        with torch.no_grad():
            #### Task 4.1
            #### Attention matrix
            #### Put the attention matrix into the list
            attentions.append(None)
            ####
    return hook

# register hook
handles = []
for idx, module in enumerate(list(model.blocks.children())):
    handles.append(module.attn.scaled_dot_attention.register_forward_hook(get_attn_softmax(f"attn{idx}")))

# forward
raw_image, inputs = transfer_single_img_to_tensor('images/cifar_example.jpg')
model.eval()
with torch.no_grad():
    logits = model(inputs)

# remove hook
for h in handles:
    h.remove()
handles = []

In [ ]:
# Check the shape of attention matrices
for attn in attentions:
    print(attn.shape)

In [ ]:
#### Task 4.1
#### Attention matrix
#### Visualize all attention matrices
#### Make sure your understand each line of this part
attention = attentions[3][0].detach().cpu()
grid_size = int(np.sqrt(attention.size(-1)))
mask = attention[1:].reshape(grid_size, grid_size).detach().numpy()
####

mask = np.array(Image.fromarray(mask / mask.max()).resize((32, 32), Image.BICUBIC))[:, :, None]
result = (mask * raw_image).astype("uint8")

fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(4, 4))

ax1.set_title('Image')
ax1.axis('off')
ax2.set_title('Attention Map')
ax2.axis('off')
ax1.imshow(raw_image)
ax2.imshow(result)

In [ ]:
#### Show probabilities
probs = F.softmax(logits, dim=-1)
top = torch.argsort(probs, dim=-1, descending=True)
for idx in top[0]:
    print('prob: {:.5f}, label: {}'.format(probs[0, idx.item()], class_names[idx.item()]))